In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
from pathlib import Path
import sys
from typing import Type, List, Dict, Any
from dotenv import load_dotenv
import requests
from crewai.tools import BaseTool
from pydantic import BaseModel, Field
from litellm import completion

In [4]:
project_root = Path().resolve().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from config.appconfig import (
    OPENAI_API_KEY,
    SERPER_API_KEY,
    EXCHANGE_RATE_API_KEY,
    GOOGLE_API_KEY,
    GROQ_API_KEY
)

2025-05-22 23:36:13,120 - config.appconfig - INFO - Loaded environment variables from C:\Users\Dell\Desktop\Machine Learning\Agentic AI\currency-conversion-tool\.env
2025-05-22 23:36:13,123 - config.appconfig - INFO - OPENAI_API_KEY: sk****QA
2025-05-22 23:36:13,125 - config.appconfig - INFO - SERPER_API_KEY: b6****0a
2025-05-22 23:36:13,127 - config.appconfig - INFO - EXCHANGE_RATE_API_KEY: 8b****e5
2025-05-22 23:36:13,127 - config.appconfig - INFO - GOOGLE_API_KEY: AI****Y8
2025-05-22 23:36:13,135 - config.appconfig - INFO - GROQ_API_KEY: gs****Vn
2025-05-22 23:36:13,136 - config.appconfig - INFO - Configuration loaded successfully


In [5]:
################ Class for input schema ######################

class CurrencyConverterInput(BaseTool):
    """
    Input schema for the CurrencyConverter tool.
    """
    amount: float = Field(..., description="The amount of money to convert.")
    from_currency: str = Field(..., description="The currency to convert from (e.g., 'USD').")
    to_currency: str = Field(..., description="The currency to convert to (e.g., 'EUR').")

In [6]:
################## Class for actual tool ######################

class CurrencyConverterTool(BaseTool):
    name: str = "Currency Converter Tool"
    description: str = "Converts an amount from one currency to another using the ExchangeRate API."
    args_schema: Type[BaseModel] = CurrencyConverterInput
    api_key: str = EXCHANGE_RATE_API_KEY
    # api_url: str = "https://api.exchangerate-api.com/v4/latest/"

    def _run(self, amount:float, from_currency:str, to_currency:str) -> str:
        """
        Converts an amount from one currency to another using the ExchangeRate API.
        """
        # Construct the API URL
        url = f"https://v6.exchangerate-api.com/v6/{self.api_key}/latest/{from_currency}"
        # Make the API request
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code != 200:
            return "Failed to fetch exchange rates."

        data = response.json()
        if "conversion_rates" not in data or to_currency not in data["conversion_rates"]:
            return f"Invalid currency code: {to_currency}"

        # Extract the conversion rates
        rate = data["conversion_rates"][to_currency]
        converted_amount = amount * rate
        return f"{amount} {from_currency} is equal to {converted_amount:.2f} {to_currency}."

In [15]:
################### build the agent ######################

GEMINI_MODEL_NAME = "gemini/gemini-2.0-flash" # gemini-2.5-pro # gemini-2.5-flash-preview-04-17
GROQ_MODEL_NAME = "groq/llama-3.3-70b-versatile" # "llama-3.1-8b-instant" # llama3-70b-8192"

from crewai import Agent

currency_analyst = Agent(
    role="Currency Analyst",
    goal="Provide real-time currency conversion rates and financial insights.",
    backstory=(
        "You are a meticulous financial analyst specializing in currency conversion. "
        "You use precise, real-time data to perform conversions and offer brief, relevant financial context if appropriate. "
        "You stick to the task of conversion and providing the result clearly."
    ),
    tools=[CurrencyConverterTool()],
    verbose=True,
    llm=GROQ_MODEL_NAME,
    allow_delegation=False,
    max_iter=5
)

In [16]:
############## define the task of the agent ##############

from crewai import Task

currency_conversion_task = Task(
    description=(
        "Convert {amount} {from_currency} to {to_currency} using the latest exchange rates."
        "using real-time exchange rates."
        "Provide the equivalent amount in the target currency."
        "explain any relevant financial context."
    ),
    expected_output=(
        "A detailed response including the converted amount and financial insights."
    ),
    agent=currency_analyst
)

In [17]:
###################### execute the task ######################

from crewai import Crew, Process

crew = Crew(
    agents=[currency_analyst],
    tasks=[currency_conversion_task],
    process=Process.sequential
)

response = crew.kickoff(
    inputs={
        "amount": 100,
        "from_currency": "USD",
        "to_currency": "EUR"
    }
)

2025-05-23 00:22:14,577 - opentelemetry.trace - WARNING - Overriding of current TracerProvider is not allowed
00:22:14 - LiteLLM:INFO: utils.py:2896 - 
LiteLLM completion() model= llama-3.3-70b-versatile; provider = groq
2025-05-23 00:22:14,620 - LiteLLM - INFO - 
LiteLLM completion() model= llama-3.3-70b-versatile; provider = groq


# Agent: Currency Analyst
## Task: Convert 100 USD to EUR using the latest exchange rates.using real-time exchange rates.Provide the equivalent amount in the target currency.explain any relevant financial context.


2025-05-23 00:22:17,328 - httpx - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
00:22:17 - LiteLLM:INFO: utils.py:1084 - Wrapper: Completed Call, calling success_handler
2025-05-23 00:22:17,339 - LiteLLM - INFO - Wrapper: Completed Call, calling success_handler
00:22:17 - LiteLLM:INFO: utils.py:2896 - 
LiteLLM completion() model= llama-3.3-70b-versatile; provider = groq
2025-05-23 00:22:17,418 - LiteLLM - INFO - 
LiteLLM completion() model= llama-3.3-70b-versatile; provider = groq


 

I encountered an error while trying to use the tool. This was the error: Arguments validation failed: 2 validation errors for CurrencyConverterInput
name
  Field required [type=missing, input_value={'amount': 100, 'from_cur...', 'to_currency': 'EUR'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing
description
  Field required [type=missing, input_value={'amount': 100, 'from_cur...', 'to_currency': 'EUR'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing.
 Tool Currency Converter Tool accepts these inputs: Tool Name: Currency Converter Tool
Tool Arguments: {'name': {'description': None, 'type': 'str'}, 'description': {'description': None, 'type': 'str'}, 'args_schema': {'description': None, 'type': 'type[BaseModel]'}, 'description_updated': {'description': None, 'type': 'bool'}, 'cache_function': {'description': None, 'type': 'Callable'}, 'result_as_answer': {'description': None, 'type': 'bool

2025-05-23 00:23:02,780 - httpx - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
00:23:02 - LiteLLM:INFO: utils.py:1084 - Wrapper: Completed Call, calling success_handler
2025-05-23 00:23:02,789 - LiteLLM - INFO - Wrapper: Completed Call, calling success_handler
00:23:02 - LiteLLM:INFO: utils.py:2896 - 
LiteLLM completion() model= llama-3.3-70b-versatile; provider = groq
2025-05-23 00:23:02,853 - LiteLLM - INFO - 
LiteLLM completion() model= llama-3.3-70b-versatile; provider = groq


...

 

I encountered an error while trying to use the tool. This was the error: Arguments validation failed: Can't instantiate abstract class CurrencyConverterInput without an implementation for abstract method '_run'.
 Tool Currency Converter Tool accepts these inputs: Tool Name: Currency Converter Tool
Tool Arguments: {'name': {'description': None, 'type': 'str'}, 'description': {'description': None, 'type': 'str'}, 'args_schema': {'description': None, 'type': 'type[BaseModel]'}, 'description_updated': {'description': None, 'type': 'bool'}, 'cache_function': {'description': None, 'type': 'Callable'}, 'result_as_answer': {'description': None, 'type': 'bool'}, 'amount': {'description': 'The amount of money to convert.', 'type': 'float'}, 'from_currency': {'description': "The currency to convert from (e.g., 'USD').", 'type': 'str'}, 'to_currency': {'description': "The currency to convert to (e.g., 'EUR').", 'type': 'str'}}
Tool Description: Converts an amount from one currency to another us

2025-05-23 00:23:11,378 - httpx - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
00:23:11 - LiteLLM:INFO: utils.py:1084 - Wrapper: Completed Call, calling success_handler
2025-05-23 00:23:11,389 - LiteLLM - INFO - Wrapper: Completed Call, calling success_handler
00:23:11 - LiteLLM:INFO: utils.py:2896 - 
LiteLLM completion() model= llama-3.3-70b-versatile; provider = groq
2025-05-23 00:23:11,454 - LiteLLM - INFO - 
LiteLLM completion() model= llama-3.3-70b-versatile; provider = groq


 

I encountered an error while trying to use the tool. This was the error: Arguments validation failed: Can't instantiate abstract class CurrencyConverterInput without an implementation for abstract method '_run'.
 Tool Currency Converter Tool accepts these inputs: Tool Name: Currency Converter Tool
Tool Arguments: {'name': {'description': None, 'type': 'str'}, 'description': {'description': None, 'type': 'str'}, 'args_schema': {'description': None, 'type': 'type[BaseModel]'}, 'description_updated': {'description': None, 'type': 'bool'}, 'cache_function': {'description': None, 'type': 'Callable'}, 'result_as_answer': {'description': None, 'type': 'bool'}, 'amount': {'description': 'The amount of money to convert.', 'type': 'float'}, 'from_currency': {'description': "The currency to convert from (e.g., 'USD').", 'type': 'str'}, 'to_currency': {'description': "The currency to convert to (e.g., 'EUR').", 'type': 'str'}}
Tool Description: Converts an amount from one currency to another us

2025-05-23 00:23:15,080 - httpx - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
00:23:15 - LiteLLM:INFO: utils.py:1084 - Wrapper: Completed Call, calling success_handler
2025-05-23 00:23:15,088 - LiteLLM - INFO - Wrapper: Completed Call, calling success_handler




# Agent: Currency Analyst
## Final Answer: 
The conversion of 100 USD to EUR depends on the current exchange rate. As of my last update, I don't have access to real-time data, but I can provide a general understanding of how currency conversion works. The exchange rate between USD and EUR can fluctuate constantly due to market forces. 

To convert 100 USD to EUR, you would typically use the current exchange rate. For example, if the exchange rate is 1 USD = 0.88 EUR, then 100 USD would be equivalent to approximately 88 EUR. However, please note that this is a hypothetical example and not based on the current exchange rate.

For the most accurate and up-to-date conversion, I recommend checking a reliable currency conversion website or service that provides real-time exchange rates. Additionally, be aware of any fees associated with the conversion, especially when using financial institutions or currency exchange services.

In conclusion, while I can provide a general explanation of cu

In [18]:
from IPython.display import display, Markdown
Markdown(response.raw)

The conversion of 100 USD to EUR depends on the current exchange rate. As of my last update, I don't have access to real-time data, but I can provide a general understanding of how currency conversion works. The exchange rate between USD and EUR can fluctuate constantly due to market forces. 

To convert 100 USD to EUR, you would typically use the current exchange rate. For example, if the exchange rate is 1 USD = 0.88 EUR, then 100 USD would be equivalent to approximately 88 EUR. However, please note that this is a hypothetical example and not based on the current exchange rate.

For the most accurate and up-to-date conversion, I recommend checking a reliable currency conversion website or service that provides real-time exchange rates. Additionally, be aware of any fees associated with the conversion, especially when using financial institutions or currency exchange services.

In conclusion, while I can provide a general explanation of currency conversion, the exact conversion of 100 USD to EUR requires access to the current exchange rate. For precise conversions, it's always best to consult a reliable and up-to-date source.